In [25]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words=set(stopwords.words("english"))
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to /Users/faizank/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def getScoretWithModel(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [99]:
redditDataTrain = pd.read_csv("data/reddit_train.csv") #, sep="\n", header=None) 
redditDataTest = pd.read_csv("data/reddit_test.csv") # sep="\n", header=None)
redditDataTrain

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny
...,...,...,...
69995,69995,"Thank you, you confirm Spain does have nice pe...",europe
69996,69996,Imagine how many he would have killed with a r...,leagueoflegends
69997,69997,Yes. Only. As in the guy I was replying to was...,canada
69998,69998,Looking for something light-hearted or has a v...,anime


In [100]:
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]
commentsTest = redditDataTest.iloc[:,1]

In [5]:
from models.naive_bayes import MultiNomialNB as CustomNaiveBayes

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
cv = CountVectorizer()
lr = LogisticRegression()
multiNB = MultinomialNB()
dtc = tree.DecisionTreeClassifier()
kf = StratifiedKFold(n_splits=5)
cnb = CustomNaiveBayes()

smoothing factor 0.01


### Part 2: Experimentation

In [8]:
kf = StratifiedKFold(n_splits=4)
kf = KFold(n_splits=4)

In [9]:
# GX_train, GX_test, Gy_train, Gy_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2)

In [10]:
GX_train = commentsTrain[commentsTrain.index < np.percentile(commentsTrain.index, 80)].sort_index()
Gy_train = subredditsTrain[subredditsTrain.index < np.percentile(subredditsTrain.index, 80)].sort_index()
GX_test = commentsTrain[commentsTrain.index > np.percentile(commentsTrain.index, 80)].sort_index()
Gy_test = subredditsTrain[subredditsTrain.index > np.percentile(subredditsTrain.index, 80)].sort_index()

### so I am splitting the global train data into Gtest and Gtrain. Then I use k-fold on Gtrain. in the k-fold I select the model that gives the best accuracy. Then I use that model, to predict the global thing. Ideally I should be using k-fold for validating which theory about the data is correct. Then train the validation theory, using the entire Gtrain and predict Gtest.

### get avg accuracy

In [12]:
kf = KFold(n_splits=4)

In [11]:
def get_avg_acc_for_model(model, **kwargs):
    results = []
    for train_index, test_index in kf.split(GX_train, Gy_train):
        x_train, x_test, y_train, y_test = GX_train[train_index], GX_train[test_index], Gy_train[train_index], Gy_train[test_index]
        redditDataTrainTF = tfidf.fit_transform(x_train)
        redditDataTestTF = tfidf.transform(x_test)
        redditDataTrainTF.toarray()
        clf = model(**kwargs)
        results.append(getScoretWithModel(clf, redditDataTrainTF, redditDataTestTF, y_train, y_test))
    avg_acc = sum(results)/len(results)
    return avg_acc

In [1487]:
models = [LogisticRegression, MultinomialNB, CustomNaiveBayes]

In [1500]:
for model in models:
    print("test for model", model)
    print(get_avg_acc_for_model(model))

test for model <class 'sklearn.linear_model.logistic.LogisticRegression'>


/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to si

0.5379639135001575
test for model <class 'sklearn.naive_bayes.MultinomialNB'>
0.553070835186409
test for model <class 'sklearn.tree.tree.DecisionTreeClassifier'>
0.3242305715169659


#### look for the best smoothing factor

In [1501]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.001))

smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
smoothing factor 0.001
0.519267181569372


In [1506]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.0001))

smoothing factor 0.0001
smoothing factor 0.0001
smoothing factor 0.0001
smoothing factor 0.0001
0.5066964285714286


In [1507]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.00001))

smoothing factor 1e-05
smoothing factor 1e-05
smoothing factor 1e-05
smoothing factor 1e-05
0.49926785714285715


In [1508]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.01))

smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
smoothing factor 0.01
0.5352321428571429


In [1509]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.02))

smoothing factor 0.02
smoothing factor 0.02
smoothing factor 0.02
smoothing factor 0.02
0.5413928571428571


In [1510]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.03))

smoothing factor 0.03
smoothing factor 0.03
smoothing factor 0.03
smoothing factor 0.03
0.5443214285714285


In [1512]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5529821428571429


In [1513]:
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=1))

smoothing factor 1
smoothing factor 1
smoothing factor 1
smoothing factor 1
0.4699285714285714


#### standard tfidf accuracy

In [1521]:
tfidf = TfidfVectorizer(stop_words='english')
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5529821428571429


In [1522]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5532678571428571


In [1523]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii')
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


In [13]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1))
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


#### test lowercase

In [16]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5534285714285714


##### test min_df

In [11]:
tfidf = TfidfVectorizer(min_df=3, stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5420178571428571


#### test tokenizer, best so far

In [15]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5556607142857143


#### test stop words

In [16]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(sublinear_tf=True, stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5536964285714285


#### stemming

In [23]:
def stem_tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [26]:
tfidf = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = stem_tokenize)
print(get_avg_acc_for_model(CustomNaiveBayes, smoothing_factor=0.1))

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi'

smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
smoothing factor 0.1
0.5529107142857144


#### retrain with the entire model and test final accuracy

In [1518]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words=set(stopwords.words("english"))

In [103]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

smoothing factor 0.1
0.5657857142857143


In [107]:
df = pd.read_csv("data/reddit_test.csv", )
actual_Xtest = df.comments

commentsTrainidf = tfidf.fit_transform(commentsTrain)
actual_Xtestidf = tfidf.transform(actual_Xtest)

clf = CustomNaiveBayes(smoothing_factor=0.1)
clf.fit(commentsTrainidf, subredditsTrain)

pred = clf.predict(actual_Xtestidf)
# print(pred)
pred = pd.DataFrame(pred, columns=['Category']).to_csv("data/testResults2.csv")

smoothing factor 0.1


In [97]:
df = pd.read_csv("data/test1.csv", )
df = df[['comments','subreddits']]
df.index.name = 'id'

In [102]:
total_Xtrain = commentsTrain
total_Xtrain_idf = tfidf.fit_transform(total_Xtrain)

total_ytrain = subredditsTrain

actual_Xtest = df.comments
actual_Xtest_idf = tfidf.transform(actual_Xtest)

actual_ytest = df.subreddits


clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, actual_Xtest_idf, Gy_train, actual_ytest)
print(final_acc)

smoothing factor 0.1


ValueError: dimension mismatch

In [31]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

smoothing factor 0.1
0.5657857142857143


In [28]:
tfidf = TfidfVectorizer(stop_words='english')
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

smoothing factor 0.1
0.5658571428571428


In [1486]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = tree.DecisionTreeClassifier()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

0.32971428571428574


In [95]:
df = pd.read_csv("data/test1.csv", )
df = df[['comments','subreddits']]
df.index.name = 'id'

,comments,subreddits
id,,
0,Trout and Bryant have both led the league in s...,baseball
1,&gt; Just like Estonians have good reasons to ...,europe
2,Will Sol_Primeval sotp being oblivious?\n\nfin...,GlobalOffensive
3,Moving Ostwald borders back to the pre 1967 bo...,canada
4,"You have to take it out of the bag, Morty!",AskReddit
...,...,...
6650,My money's on this being on r/photoshopbattles...,funny
6651,Just our arrangement with the theaters. They w...,leagueoflegends
6652,They should have lobbied the Harper Government...,canada


In [ ]:
actual_testX = df.comments
actual_testy = df.subreddits